[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Simone-Alghisi/HMD-Lab/blob/master/notebooks/1_introduction.ipynb)

On Colab:
1. Switch to a GPU Runtime by clicking on *Runtime > Change runtime type > T4 GPU*
2. Run the cell below

In [ ]:
# For Google Colab only
!git clone https://github.com/Simone-Alghisi/HMD-Lab.git
%cd /content/HMD-Lab/notebooks 

# Introduction

## Running the code
After [installing the required components](./0_installation.ipynb), you can interact with the project using:
```shell
python -m main
```

## Prompting (In-Context Learning)

A prompt is natural language sequence to condition the generation of the model (prefixes for the Language Model).

We Zero-Shot, One-Shot, and Few-Shot prompt the model to obtain better results.

In [ ]:
import sys

sys.path.append("..")

from utils import MODELS
from transformers import AutoTokenizer

model_name, InitModel, prepare_text = MODELS["qwen3"]

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = InitModel(
    model_name,
    dtype="auto",
    device_map="cuda:0",
)

/home/simone/miniconda3/envs/hmd/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.97it/s]


In [ ]:
import torch

from notebooks.notebook_utils import display_conversation
from models.qwen3 import prepare_text


task_prompt = "Answer using as few words as possible. Answer to this question by considering the Examples provided."
user_message = "What's 6+6?"

### Zero-Shot

Usually, Zero-Shot prompts contain 
1. (optionally) a description of the task, which should specify the model what to do (e.g., *"Summarize this text"*)
2. the input for the current model (e.g., the summary)

In [ ]:
zero_shot = [
    {
        "role": "system", 
        "content": task_prompt
    }
]

text = prepare_text(user_message, tokenizer, zero_shot)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

with torch.no_grad():
    generated_ids = model.generate(**model_inputs, max_new_tokens=16384).cpu()

# decode the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :].tolist()
content = tokenizer.decode(output_ids, skip_special_tokens=True)
display_conversation(zero_shot, user_message, content)

### Conversation

**System:** Answer using as few words as possible. Answer to this question by considering the Examples provided.

**User:** What's 6+6?

**Assistant:** 12

### One-Shot

One-Shot prompts can be seen as an extension of Zero-Shot, where we also include an example for the task.

In [25]:
one_shot = [
    {
        "role": "system", 
        "content": task_prompt + "\n\nExample: What's 4+4? 8"
    },
]

text = prepare_text(user_message, tokenizer, one_shot)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

with torch.no_grad():
    generated_ids = model.generate(**model_inputs, max_new_tokens=16384).cpu()

# decode the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :].tolist()
content = tokenizer.decode(output_ids, skip_special_tokens=True)

display_conversation(one_shot, user_message, content)

### Conversation

**System:** Answer using as few words as possible. Answer to this question by considering the Examples provided.

Example: What's 4+4? 8

**User:** What's 6+6?

**Assistant:** 12

### Few-Shots

A Few-Shots prompt can be seen as a One-Shot prompt, with more than one example.

In [28]:
few_shot = [
    {
        "role": "system", 
        "content": task_prompt + "\n\nExample:\n- What's 4+4? 6\n- What's 1+1? 6\n- What's 2+2? 6\n- What's 3+3? 10"
    },
]

text = prepare_text(user_message, tokenizer, few_shot)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

with torch.no_grad():
    generated_ids = model.generate(**model_inputs, max_new_tokens=16384).cpu()

# decode the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :].tolist()
content = tokenizer.decode(output_ids, skip_special_tokens=True)

display_conversation(few_shot, user_message, content)

### Conversation

**System:** Answer using as few words as possible. Answer to this question by considering the Examples provided.

Example:
- What's 4+4? 6
- What's 1+1? 6
- What's 2+2? 6
- What's 3+3? 10

**User:** What's 6+6?

**Assistant:** 6

## Classification with LLMs

LLMs can be used as zero-shot or few-shot classification by prompting them to return a label or short answer. This approach is convenient for quick experiments or when labeled data is scarce, but keep in mind: outputs may require post-processing to map natural-language answers to fixed labels.

Considerations:
- Prefer short, explicit answer formats (e.g., "Answer: <label>").
- Use few-shot examples to disambiguate label wording.
- Validate on a held-out set and apply simple normalization (lowercasing, trimming) to model outputs.

In [32]:
text_to_classify = "I really enjoyed the movie. The acting was superb."
messages = [
    {
        "role": "system", 
        "content": "You are a concise classifier. Classify the sentiment of the following text. Choose one label: positive, negative"
    }
]
user_message = "Text: \"" + text_to_classify + "\"\nAnswer:"

text = prepare_text(user_message, tokenizer, messages)
inputs = tokenizer([text], return_tensors="pt").to(model.device)
with torch.no_grad():
    generated = model.generate(**inputs, max_new_tokens=32).cpu()

output_ids = generated[0][len(inputs.input_ids[0]) :].tolist()
out = tokenizer.decode(output_ids, skip_special_tokens=True)

# Simple normalization to match labels
out = out.strip().lower().split()[0]
display_conversation(messages, user_message, out)

### Conversation

**System:** You are a concise classifier. Classify the sentiment of the following text. Choose one label: positive, negative

**User:** Text: "I really enjoyed the movie. The acting was superb."
Answer:

**Assistant:** positive

## Data generation with LLMs

LLMs are useful to generate synthetic data for training or augmenting datasets. Use prompts that explicitly request the format you want (JSON, CSV-like lines, or label + text). Validate generated samples and filter or deduplicate before using them for training.

In [47]:
import json
from tqdm import trange

def gen_sample(prompt, sentiment):
    messages = [
        {
            "role":"system",
            "content":"You output a single JSON object with keys 'text' and 'label'."
        }
    ]
    text = prepare_text(prompt.format(sentiment), tokenizer, messages)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        ids = model.generate(**inputs, max_new_tokens=128).cpu()
    out_ids = ids[0][len(inputs.input_ids[0]):].tolist()
    return tokenizer.decode(out_ids, skip_special_tokens=True)

prompt_template = (
    "Generate a short movie review and a sentiment label. Output exactly one JSON object like:\n"
    "{{\"text\": \"...\", \"label\": \"{}\"}}\n\n"

)

n = 4
samples = []
for i in trange(n):
    sentiment = "positive" if i%2==0 else "negative"
    raw = gen_sample(prompt_template, sentiment)
    try:
        obj = json.loads(raw.strip())
        samples.append(obj)
    except Exception:
        print("Failed to parse generated sample:\n", raw)

for s in samples:
    print(json.dumps(s, ensure_ascii=False, indent=2))

100%|██████████| 4/4 [00:06<00:00,  1.54s/it]

100%|██████████| 4/4 [00:06<00:00,  1.54s/it]

{
  "text": "This film is a breathtaking blend of stunning visuals, compelling storytelling, and powerful performances. The director's vision comes through clearly, and the emotional depth resonates long after the credits roll.",
  "label": "positive"
}
{
  "text": "The film suffers from a lackluster plot and underdeveloped characters, making it difficult to stay engaged throughout. The pacing is sluggish, and the dialogue feels forced and unnatural.",
  "label": "negative"
}
{
  "text": "This film is a breathtaking blend of emotion and stunning visuals. The storytelling is compelling, and the performances are heartfelt and authentic. A must-watch for anyone who loves deeply moving cinema.",
  "label": "positive"
}
{
  "text": "The film suffers from poor pacing and uninspired storytelling, making it difficult to stay engaged throughout. The characters feel flat, and the plot lacks any meaningful development.",
  "label": "negative"
}


## End-to-end OrderBot

You will now modify the code in  [main.py](../main.py) to act as an End-to-End OrderBot.

1. You can change the content on the file by using `nano main.py` on your remote machine
2. Change the "content" of the first message to include the prompt below
3. Interact with the system to understand its capabilities (`python -m main`) 

In [ ]:
"""You are OrderBot, an automated service to collect orders for a pizza restaurant.
You first greet the customer, then collects the order,
and then asks if it's a pickup or delivery.
You wait to collect the entire order, then summarize it and check for a final
time if the customer wants to add anything else.
If it's a delivery, you ask for an address.
Finally you collect the payment.
Make sure to clarify all options, extras and sizes to uniquely
identify the item from the menu.
You respond in a short, very conversational friendly style.
The menu includes:
pepperoni pizza 12.95, 10.00, 7.00
cheese pizza 10.95, 9.25, 6.50
eggplant pizza 11.95, 9.75, 6.75
fries 4.50, 3.50
greek salad 7.25
Toppings: \\ extra cheese 2.00, \\ mushrooms 1.50 \\ sausage 3.00 \\ canadian bacon 3.50
AI sauce 1.50 \\ peppers 1.00
Drinks: \\ coke 3.00, 2.00, 1.00 \\ sprite 3.00, 2.00, 1.00 \\ bottled water 5.00
"""